# Carregando bibliotecas

In [ ]:
!pip install netCDF4
!pip install cfgrib

In [ ]:
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

# Importando dados

In [ ]:
#################################################################
# IMPORTANDO DADOS DO ERA5 --> Registros de vento, pressão atmosférica, temperatura do ar e temperatura do ponto de orvalho
#################################################################
ds_ERA5 = xr.open_dataset('/content/ERA5_2015-2024.grib', engine='cfgrib')
print(ds_ERA5)

In [ ]:
###################################################
# IMPORTANDO DADOS DO MERGE-CPTEC --> Registros de Chuva
###################################################
ds_MERGE = xr.open_dataset('/content/MERGE_CPTEC_BRASIL_2015-2024.nc')
print(ds_MERGE)

In [ ]:
# Calculando a Velocidade do Vento a 10metros
ds_ERA5['vv10m'] = np.sqrt(ds_ERA5['v10']**2 + ds_ERA5['u10']**2)

# Calculando a direção do vento
ds_ERA5['dv10m'] = (np.rad2deg(np.arctan2(-ds_ERA5['u10'], -ds_ERA5['v10'])) + 360) % 360

# Conversão da temperatura do ar de Kelvin para Celsius
ds_ERA5['temp_c'] = ds_ERA5['t2m'] - 273.15

# Conversão da temperatura do ponto de orvalho de Kelvin para Celsius
ds_ERA5['temp_orvalho_c'] = ds_ERA5['d2m'] - 273.15

# Conversão da pressão atmosférica de Pa para hPa
ds_ERA5['pressao_atm'] = ds_ERA5['sp']/100

# Calculando a pressão de vapor de saturação e pressão de vapor real
T = ds_ERA5['temp_c']
Td = ds_ERA5['temp_orvalho_c']

es = 6.112 * np.exp((17.67 * T) / (T + 243.5))
e = 6.112 * np.exp((17.67 * Td) / (Td + 243.5))

# Calcula a Umidade Relativa
ds_ERA5['umidade_rel'] = 100 * (e / es)

# Séries temporais

In [ ]:
###################################################
######## DICIONÁRIO DE VARIAVÉIS ##################

lat = -9.55   # sua latitude
lon = -35.77   # sua longitude
cidade = 'Maceió/AL'
ano1 = '2019'
ano2 = '2022'
###################################################
###################################################

# Seleciona o ponto mais próximo dessa coordenada
ERA5_ponto = ds_ERA5.sel(latitude=lat, longitude=lon, method='nearest')
MERGE_ponto = ds_MERGE.sel(lat=lat, lon=lon, method='nearest')

In [ ]:
# ========================
# GERANDO RESULTADOS PARA TEMPERATURA
# ========================

# Extraindo a série temporal
serie = ERA5_ponto['temp_c']

# Converte para DataFrame e adiciona colunas auxiliares
df = serie.to_dataframe().reset_index()
df['time'] = pd.to_datetime(df['time'])
df['Ano'] = df['time'].dt.year
df['Mes'] = df['time'].dt.month

# Média mensal climatológica (todos os anos)
climatologia = df.groupby('Mes')['temp_c'].mean()

# Criar um dicionário para armazenar os anos específicos
anos_especificos = {}

# Média mensal para os anos desejados
anos_especificos[ano1] = df[df['Ano'] == int(ano1)].groupby('Mes')['temp_c'].mean()
anos_especificos[ano2] = df[df['Ano'] == int(ano2)].groupby('Mes')['temp_c'].mean()

# Garante que todos os meses estejam presentes (1 a 12)
meses = np.arange(1, 13)
climatologia = climatologia.reindex(meses)
anos_especificos[ano1] = anos_especificos[ano1].reindex(meses)
anos_especificos[ano2] = anos_especificos[ano2].reindex(meses)

# Largura e deslocamento das barras
bar_width = 0.25
x = np.arange(1, 13)

# Plotando
plt.figure(figsize=(14, 6))

plt.bar(x - bar_width, climatologia.values, width=bar_width, label='Climatologia', color='skyblue')
plt.bar(x, anos_especificos[ano1].values, width=bar_width, label=ano1, color='orange')
plt.bar(x + bar_width, anos_especificos[ano2].values, width=bar_width, label=ano2, color='green')

# Rótulos e estilo
plt.xticks(x, ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun',
               'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'])
plt.ylabel("Temperatura do Ar [°C]", fontsize=12)
plt.ylim(0, 35)
plt.title(f"Temperatura Média Mensal - {cidade}", fontsize=14, fontweight='bold')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend(fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# ========================
# GERANDO RESULTADOS PARA VELOCIDADE DO VENTO
# ========================

# Extraindo a série temporal
serie = ERA5_ponto['vv10m']

# Converte para DataFrame e adiciona colunas auxiliares
df = serie.to_dataframe().reset_index()
df['time'] = pd.to_datetime(df['time'])
df['Ano'] = df['time'].dt.year
df['Mes'] = df['time'].dt.month

# Média mensal climatológica (todos os anos)
climatologia = df.groupby('Mes')['vv10m'].mean()

# Criar um dicionário para armazenar os anos específicos
anos_especificos = {}

# Média mensal para os anos desejados
anos_especificos[ano1] = df[df['Ano'] == int(ano1)].groupby('Mes')['vv10m'].mean()
anos_especificos[ano2] = df[df['Ano'] == int(ano2)].groupby('Mes')['vv10m'].mean()

# Garante que todos os meses estejam presentes (1 a 12)
meses = np.arange(1, 13)
climatologia = climatologia.reindex(meses)
anos_especificos[ano1] = anos_especificos[ano1].reindex(meses)
anos_especificos[ano2] = anos_especificos[ano2].reindex(meses)

# Largura e deslocamento das barras
bar_width = 0.25
x = np.arange(1, 13)

# Plotando
plt.figure(figsize=(14, 6))

plt.bar(x - bar_width, climatologia.values, width=bar_width, label='Climatologia', color='skyblue')
plt.bar(x, anos_especificos[ano1].values, width=bar_width, label=ano1, color='orange')
plt.bar(x + bar_width, anos_especificos[ano2].values, width=bar_width, label=ano2, color='green')

# Rótulos e estilo
plt.xticks(x, ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun',
               'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'])
plt.ylabel("Velocidade do Vento [m/s]", fontsize=12)
plt.ylim(0, 7)
plt.title(f"Velocidade do Vento Média Mensal - {cidade}", fontsize=14, fontweight='bold')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend(fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# ========================
# GERANDO RESULTADOS PARA PRECIPITAÇÃO
# ========================

# Extraindo a série temporal
serie = MERGE_ponto['pacum']

# Converte para DataFrame e adiciona colunas auxiliares
df = serie.to_dataframe().reset_index()
df['time'] = pd.to_datetime(df['time'])
df['Ano'] = df['time'].dt.year
df['Mes'] = df['time'].dt.month

# Média mensal climatológica (todos os anos)
climatologia = df.groupby('Mes')['pacum'].mean()

# Criar um dicionário para armazenar os anos específicos
anos_especificos = {}

# Média mensal para os anos desejados
anos_especificos[ano1] = df[df['Ano'] == int(ano1)].groupby('Mes')['pacum'].mean()
anos_especificos[ano2] = df[df['Ano'] == int(ano2)].groupby('Mes')['pacum'].mean()

# Garante que todos os meses estejam presentes (1 a 12)
meses = np.arange(1, 13)
climatologia = climatologia.reindex(meses)
anos_especificos[ano1] = anos_especificos[ano1].reindex(meses)
anos_especificos[ano2] = anos_especificos[ano2].reindex(meses)

# Largura e deslocamento das barras
bar_width = 0.25
x = np.arange(1, 13)

# Plotando
plt.figure(figsize=(14, 6))

plt.bar(x - bar_width, climatologia.values, width=bar_width, label='Climatologia', color='skyblue')
plt.bar(x, anos_especificos[ano1].values, width=bar_width, label=ano1, color='orange')
plt.bar(x + bar_width, anos_especificos[ano2].values, width=bar_width, label=ano2, color='green')

# Rótulos e estilo
plt.xticks(x, ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun',
               'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'])
plt.ylabel("Precipitação [mm]", fontsize=12)
plt.ylim(0, 500)
plt.title(f"Precipitação Média Mensal - {cidade}", fontsize=14, fontweight='bold')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend(fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# ========================
# GERANDO RESULTADOS PARA ANOMALIA DE PRECIPITAÇÃO
# ========================

# Extraindo a série temporal da variável de interesse
serie = ERA5_ponto['temp_c']

# Converte para DataFrame e adiciona colunas auxiliares
df = serie.to_dataframe().reset_index()
df['time'] = pd.to_datetime(df['time'])
df['Ano'] = df['time'].dt.year
df['Mes'] = df['time'].dt.month

# Converte ano1 e ano2 para inteiros
ano1_int = int(ano1)
ano2_int = int(ano2)

# Climatologia mensal (média de todos os anos)
climatologia = df.groupby('Mes')['temp_c'].mean()

# Médias mensais dos anos específicos
media_ano1 = df[df['Ano'] == ano1_int].groupby('Mes')['temp_c'].mean()
media_ano2 = df[df['Ano'] == ano2_int].groupby('Mes')['temp_c'].mean()

# Garante que todos os meses estejam presentes
meses = np.arange(1, 13)
climatologia = climatologia.reindex(meses)
media_ano1 = media_ano1.reindex(meses)
media_ano2 = media_ano2.reindex(meses)

# Cálculo das anomalias
anomalia_ano1 = media_ano1 - climatologia
anomalia_ano2 = media_ano2 - climatologia

# =============================================
# PLOTANDO AS ANOMALIAS
# =============================================

# Configurações do gráfico
bar_width = 0.3
x = np.arange(1, 13)

plt.figure(figsize=(14, 6))
plt.bar(x - bar_width/2, anomalia_ano1, width=bar_width, label=f'Anomalia {ano1}', color='coral')
plt.bar(x + bar_width/2, anomalia_ano2, width=bar_width, label=f'Anomalia {ano2}', color='seagreen')

# Estilização
plt.axhline(0, color='black', linewidth=1)
plt.xticks(x, ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun',
               'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'])
plt.ylabel("Anomalia de Temperatura [°C]", fontsize=12)
plt.title(f"Anomalia Mensal de Temperatura - {cidade}", fontsize=14, fontweight='bold')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# ========================
# GERANDO RESULTADOS PARA ANOMALIA DE VELOCIDADE DO VENTO
# ========================

# Extraindo a série temporal da variável de interesse
serie = ERA5_ponto['vv10m']

# Converte para DataFrame e adiciona colunas auxiliares
df = serie.to_dataframe().reset_index()
df['time'] = pd.to_datetime(df['time'])
df['Ano'] = df['time'].dt.year
df['Mes'] = df['time'].dt.month

# Converte ano1 e ano2 para inteiros
ano1_int = int(ano1)
ano2_int = int(ano2)

# Climatologia mensal (média de todos os anos)
climatologia = df.groupby('Mes')['vv10m'].mean()

# Médias mensais dos anos específicos
media_ano1 = df[df['Ano'] == ano1_int].groupby('Mes')['vv10m'].mean()
media_ano2 = df[df['Ano'] == ano2_int].groupby('Mes')['vv10m'].mean()

# Garante que todos os meses estejam presentes
meses = np.arange(1, 13)
climatologia = climatologia.reindex(meses)
media_ano1 = media_ano1.reindex(meses)
media_ano2 = media_ano2.reindex(meses)

# Cálculo das anomalias
anomalia_ano1 = media_ano1 - climatologia
anomalia_ano2 = media_ano2 - climatologia

# =============================================
# PLOTANDO AS ANOMALIAS
# =============================================

# Configurações do gráfico
bar_width = 0.3
x = np.arange(1, 13)

plt.figure(figsize=(14, 6))
plt.bar(x - bar_width/2, anomalia_ano1, width=bar_width, label=f'Anomalia {ano1}', color='coral')
plt.bar(x + bar_width/2, anomalia_ano2, width=bar_width, label=f'Anomalia {ano2}', color='seagreen')

# Estilização
plt.axhline(0, color='black', linewidth=1)
plt.xticks(x, ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun',
               'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'])
plt.ylabel("Anomalia de Velocidade do Vento [m/s]", fontsize=12)
plt.title(f"Anomalia Mensal de Velocidade do Vento - {cidade}", fontsize=14, fontweight='bold')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# ========================
# GERANDO RESULTADOS PARA ANOMALIA DE PRECIPITAÇÃO
# ========================

# Extraindo a série temporal
serie = MERGE_ponto['pacum']

# Converte para DataFrame e adiciona colunas auxiliares
df = serie.to_dataframe().reset_index()
df['time'] = pd.to_datetime(df['time'])
df['Ano'] = df['time'].dt.year
df['Mes'] = df['time'].dt.month

# Converte ano1 e ano2 para inteiros
ano1_int = int(ano1)
ano2_int = int(ano2)

# Climatologia mensal (média de todos os anos)
climatologia = df.groupby('Mes')['pacum'].mean()

# Médias mensais dos anos específicos
media_ano1 = df[df['Ano'] == ano1_int].groupby('Mes')['pacum'].mean()
media_ano2 = df[df['Ano'] == ano2_int].groupby('Mes')['pacum'].mean()

# Garante que todos os meses estejam presentes
meses = np.arange(1, 13)
climatologia = climatologia.reindex(meses)
media_ano1 = media_ano1.reindex(meses)
media_ano2 = media_ano2.reindex(meses)

# Cálculo das anomalias
anomalia_ano1 = media_ano1 - climatologia
anomalia_ano2 = media_ano2 - climatologia

# =============================================
# PLOTANDO AS ANOMALIAS
# =============================================

# Configurações do gráfico
bar_width = 0.3
x = np.arange(1, 13)

plt.figure(figsize=(14, 6))
plt.bar(x - bar_width/2, anomalia_ano1, width=bar_width, label=f'Anomalia {ano1}', color='coral')
plt.bar(x + bar_width/2, anomalia_ano2, width=bar_width, label=f'Anomalia {ano2}', color='seagreen')

# Estilização
plt.axhline(0, color='black', linewidth=1)
plt.xticks(x, ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun',
               'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'])
plt.ylabel("Anomalia de Precipitação [mm]", fontsize=12)
plt.title(f"Anomalia Mensal de Precipitação - {cidade}", fontsize=14, fontweight='bold')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()